In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hydrant.topology.geom as gm
import subprocess
import os
from   shapely.geometry import Point
import yaml
import shutil
import warnings
import networkx as nx
import re
import copy
from   riverlakenetwork import BurnLakes, Utility
warnings.filterwarnings("ignore")

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/Users/shg096/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/traitlets/traitlets.py", line 632, in get
    value = obj._trait_values[self.name]
KeyError: '_control_lock'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/shg096/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/Users/shg096/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/ipykernel/kernelbase.py", line 301, in dispatch_control
    async with self._control_lock:
  File "/Users/shg096/Desktop/RiverLakeNetwork/env/RiverLakeEnv/lib/python3.9/site-packages/traitlets/traitlets.py", line 687, in __get__
    return t.cast(G, self.get(obj, cls))  # the G should encode the Optional
  File "/Users/shg096/Desktop/RiverLak

In [3]:
# outputfolder definition
OutFolder = '/Users/shg096/Desktop/LakeRiverOut/'

# location of merit basins bux fixed files
riv_file_template='/Users/shg096/Desktop/MERIT_Hydro/riv/riv_pfaf_*_MERIT_Hydro_v07_Basins_v01_bugfix1.shp',
cat_file_template='/Users/shg096/Desktop/MERIT_Hydro/cat/cat_pfaf_*_MERIT_Hydro_v07_Basins_v01_bugfix1.shp',
cst_file_template='/Users/shg096/Desktop/MERIT_Hydro/hill/hillslope_*_clean.shp'

# location of hydrolakes
lake_file = '/Volumes/F:/hydrography/hydrolakes/HydroLAKES_polys_v10_shp/HydroLAKES_polys_v10_shp/HydroLAKES_polys_v10.shp'

# pfaf to be computed
pfafs = ["71", "72"]



In [ ]:
#load hydrolakeDataset
lakeO = gpd.read_file(lake_file) # read the hydrolake dataset
#lakeO = lakeO.loc[lakeO["Hylak_id"] < 200].reset_index(drop=True)

for pfaf in pfafs:

    pfaf_base = f"pfaf{pfaf}"

    # create the folder if not existed
    org_folder = os.path.join(OutFolder, f"{pfaf_base}_org")
    if os.path.isdir(org_folder):
        try:
            shutil.rmtree(org_folder)
        except OSError as e:
            raise RuntimeError(f"Failed to remove {org_folder}: {e}")
    os.makedirs(org_folder, exist_ok=True)
    
    # read the pfaf merit folder
    riv, cat = Utility.merit_read_file(pfaf,
                                       riv_file_template='/Users/shg096/Desktop/MERIT_Hydro/riv/riv_pfaf_*_MERIT_Hydro_v07_Basins_v01_bugfix1.shp',
                                       cat_file_template='/Users/shg096/Desktop/MERIT_Hydro/cat/cat_pfaf_*_MERIT_Hydro_v07_Basins_v01_bugfix1.shp',
                                       cst_file_template='/Users/shg096/Desktop/MERIT_Hydro/hill/hillslope_*_clean.shp')
    # save riv and cat
    riv.to_file(os.path.join(org_folder, "riv.gpkg"))
    cat.to_file(os.path.join(org_folder, "cat.gpkg"))

    # create the folder if not existed
    corrected_folder = os.path.join(OutFolder, f"{pfaf_base}_corrected")
    if os.path.isdir(corrected_folder):
        try:
            shutil.rmtree(corrected_folder)
        except OSError as e:
            raise RuntimeError(f"Failed to remove {corrected_folder}: {e}")
    os.makedirs(corrected_folder, exist_ok=True)
    
    # create the config and pass it to the Burn lake
    config = {
        "riv": riv,
        "riv_dict": {
            "COMID": {"col":"COMID"},
            "NextDownCOMID": {"col":"NextDownID"},
            "length": {"col":"lengthkm"},
            "uparea": {"col":"uparea","unit":"km2"}
        },
        "cat": cat,
        "cat_dict": {
            "COMID": {"col":"COMID"},
            "unitarea": {"col":"unitarea","unit":"km2"},
        },
        "lake": lakeO,
        "lake_dict": {
            "LakeCOMID": {"col":"Hylak_id"},
            "unitarea": {"col":"Lake_area","unit":"km2"}
        },
    }
    
    bl = BurnLakes(config)
    bl.riv.to_file(os.path.join(corrected_folder, "riv.gpkg"))
    bl.cat.to_file(os.path.join(corrected_folder, "cat.gpkg"))
    bl.lake.to_file(os.path.join(corrected_folder, "lake.gpkg"))

    

=== Input loader started at : 2025-12-18 14:45:51  ===
riv: Loaded
riv_dict: {'COMID': {'col': 'COMID'}, 'NextDownCOMID': {'col': 'NextDownID'}, 'length': {'col': 'lengthkm'}, 'uparea': {'col': 'uparea', 'unit': 'km2'}}
cat: Loaded
cat_dict: {'COMID': {'col': 'COMID'}, 'unitarea': {'col': 'unitarea', 'unit': 'km2'}}
lake: Loaded
lake_dict: {'LakeCOMID': {'col': 'Hylak_id'}, 'unitarea': {'col': 'Lake_area', 'unit': 'km2'}}
=== Input loader finished at: 2025-12-18 14:45:52  ===
=== Input loader took      : 0:00:01.274374  ===========================
=== Input checker started at : 2025-12-18 14:45:52  ===
Subbasin and lake area units are consistent: km2
riv CRS: EPSG:4326
cat CRS: EPSG:4326
lake CRS: EPSG:4326
=== Input checker finished at: 2025-12-18 14:45:53  ===
=== Input checker took      : 0:00:01.373910  ===========================
=== Resolving lakes started at : 2025-12-18 14:45:53  ===
==== Number of lakes after subsetting: 140586 ====
